In [1]:
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, TextStreamer, LogitsProcessorList, LogitsProcessor)
import torch
from langchain_huggingface import HuggingFacePipeline
from markitdown import MarkItDown
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup
import pyhtml2md
import torch

import re


In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

2.5.1
True


In [3]:
def basic_search(query, max_results = 5):
    search_result = DDGS().text(query, max_results=max_results)
    return search_result

def surf_website(href):
    global md 

    response = requests.get(href, timeout=10)

    if response.status_code == 200:
        html_text = response.text
    else:
        print(f"Failed to fetch: {response.status_code}")
        return None
    soup = BeautifulSoup(response.text, "html.parser")
    for script in soup(["script", "noscript"]):
        script.decompose()
    cleaned_html = str(soup)
    result = pyhtml2md.convert(cleaned_html)
    return result, html_text 

def split_markdown_by_headings(md_text, max_chunk_tokens=512, tokenizer=None):
    # spilt the document by #
    sections = re.split(r"(?=^#{1,6} )", md_text, flags=re.MULTILINE)
    chunks = []

    for section in sections:
        if tokenizer:
            input_ids = tokenizer(section).input_ids
            token_len = len(input_ids)
        else:
            words = section.split()
            token_len = len(words)

        # if token length of current section is higher than max_chunk_tokens
        if token_len <= max_chunk_tokens:
            chunks.append(section.strip())
        else:
            # split the section into chunk with max_chunk_tokens
            if tokenizer:
                for i in range(0, token_len, max_chunk_tokens):
                    sub_input_ids = input_ids[i:i+max_chunk_tokens]
                    sub_chunk = tokenizer.decode(sub_input_ids, skip_special_tokens=True)
                    chunks.append(sub_chunk.strip())
            else:
                # fallback for non-tokenizer: use word split
                for i in range(0, token_len, max_chunk_tokens):
                    sub_chunk = " ".join(words[i:i+max_chunk_tokens])
                    chunks.append(sub_chunk.strip())

    return chunks

def expand_chunk_context_tokenized(chunks, tokenizer, prefix_tokens=30, suffix_tokens=30):
    expanded_chunks = []

    for i, chunk in enumerate(chunks):
        # prefix
        if i > 0:
            prev_chunk = chunks[i - 1]
            prev_input_ids = tokenizer(prev_chunk, add_special_tokens=False).input_ids
            prefix_ids = prev_input_ids[-prefix_tokens:]
            prefix = tokenizer.decode(prefix_ids, skip_special_tokens=True)
        else:
            prefix = ""
        # suffic
        if i < len(chunks) - 1:
            next_chunk = chunks[i + 1]
            next_input_ids = tokenizer(next_chunk, add_special_tokens=False).input_ids
            suffix_ids = next_input_ids[:suffix_tokens]
            suffix = tokenizer.decode(suffix_ids, skip_special_tokens=True)
        else:
            suffix = ""

        expanded = prefix + chunk + suffix
        expanded_chunks.append(expanded.strip())

    return expanded_chunks

In [4]:
# model_id = "deepcogito/cogito-v1-preview-llama-3B"
model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
streamer = TextStreamer(tokenizer, skip_prompt=True)
quantization_config = BitsAndBytesConfig(
    # load_in_4bit=True,
    # bnb_4bit_quant_type="nf4",
    load_in_8bit=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config = quantization_config
)

pipeline = pipeline(
    "text-generation",
    model=model, # use the model in the above
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id,
    do_sample = False,
    streamer=streamer,
)

C:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\ipywidgets\widgets\widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
C:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\ipywidgets\widgets\widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
C:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\ipywidgets\widgets\widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
C:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\ipywidgets\widgets\widget.py:469: DeprecationWarning: The `i

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [5]:
search = basic_search("dga domain", max_results=3)
title = search[1]['title']
mark_down = surf_website(search[1]['href'])[0]
splited_md = split_markdown_by_headings(mark_down, max_chunk_tokens=250, tokenizer=tokenizer)
chunk = expand_chunk_context_tokenized(splited_md, tokenizer=tokenizer, prefix_tokens=30, suffix_tokens=75)

In [7]:
for md in splited_md:
    print(len(tokenizer(md).input_ids))

250
107
250
251
251
251
251
252
251
251
59
118
152
133
250
101
250
84
250
25
250
44
250
20
250
72
250
251
194
250
56
250
48
250
79
188
99
250
251
251
251
251
179
249
250
249
251
251
251
47
250
251
251
251
250
251
21
250
117
250
251
251
251
251
33
160


In [17]:
DEEP_THINKING_INSTRUCTION = "Enable deep thinking subroutine."
messages = [
    {"role": "system", "content": DEEP_THINKING_INSTRUCTION},
    {"role": "system", "content": "You are a cyber security expert"},
    {"role": "user", "content": "What is dga?, please explain it briefly"},
]

outputs = pipeline(
    messages,
    max_new_tokens = 512,
    repetition_penalty=1.2
)

<think>
Okay, the user asked about DGAs and wants an explanation in brief.

First, I need to recall what DGA stands for. It's Domain Generation Algorithm, right? Used by malware creators to generate multiple domains that host malicious content or phishing sites.

I should start with defining DGA simply. Then mention how it works: using algorithms like Caesar cipher or substitution ciphers to create variations of domain names. This helps avoid detection since legitimate websites don't use such methods.

Next, highlight why DGAs are important—malware can switch between these domains quickly, making them harder to take down. Also, they're often used in botnets and ransomware campaigns.

Maybe add some examples if possible without getting too technical. But keep it concise as per the request.

Wait, did I miss anything? Oh yes, maybe note that detecting all variants requires constant monitoring and updates to blocklists. That adds context on why this technique persists despite efforts.
</t

In [91]:
splited_md[0]

'Domain generation algorithm - Wikipedia\n=======================================\n\n[Jump to content](#bodyContent)\n\n\n\t\tMain menu\n\n\nMain menu\tmove to sidebar\thide\n\n\n\t\tNavigation\n\n\n\t\t\t\t\t\t- [Main page](/wiki/Main_Page "Visit the main page [z]")\n- [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")\n- [Current events](/wiki/Portal:Current_events "Articles related to current events")\n- [Random article](/wiki/Special:'

In [7]:
title

'Threat Brief: Understanding Domain Generation Algorithms (DGA) - Unit 42'

In [9]:
# DEEP_THINKING_INSTRUCTION = "Enable deep thinking subroutine."
system_prompt = """
You will be given a README-style format text.
Title of article: Threat Brief: Understanding Domain Generation Algorithms (DGA) - Unit 42
Your task is to extract the article content from this document.
Ignore all non-article elements such as navigation menus, headers, footers, sidebars, image captions and language switchers.
If you observe there are input appears to be part of the actual article body return True.
Else return False.
"""
chunk_with_content2 = []
true_indices = []
for i, md in enumerate(splited_md, start=0):
    messages = [
        # {"role": "system", "content": DEEP_THINKING_INSTRUCTION},
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": md}
    ]
    
    print("----------start input message---------------")
    print(md)
    print("----------end input message---------------")
    outputs = pipeline(
        messages,
        max_new_tokens = 200,
        return_full_text=False,
    )
    output  = outputs[0]['generated_text']
    output  = output.strip().lower()
    if 'true' in output and 'false' not in output:
        result = True
    elif 'false' in output and 'true' not in output:
        result = False
    else:
        result = None  # ambiguous
    if result:
        chunk_with_content2.append(md)
        true_indices.append(i)

----------start input message---------------
Threat Brief: Understanding Domain Generation Algorithms (DGA)

[![Logo](/wp-content/uploads/2021/07/PANW_Parent.png)
](https://www.paloaltonetworks.com/)

[![Unit42 Logo](https://unit42.paloaltonetworks.com/wp-content/themes/unit42-v6/dist/images/unit42-logo-white.svg)
](https://unit42.paloaltonetworks.com/)

**

Menu


- [Tools](https://unit42.paloaltonetworks.com/tools/)
- [ATOMs](https://unit42.paloaltonetworks.com/atoms/)
- [Security Consulting](https://www.paloaltonetworks.com/unit42)
- [About Us](https://unit42.paloaltonetworks.com/about-unit-42/)
- [**Under Attack?**](https://start.paloaltonetworks.com/contact-unit42.html)


[English](#)


- [English](https://unit42.paloaltonetworks.com/threat-brief-understanding-domain-generation-algorithms-dga/)
- [Japanese](https://unit42.paloaltonetworks.com/ja/threat-brief
----------end input message---------------
False<|eot_id|>
----------start input message---------------
-understanding-domai

In [8]:
# for chunk in chunk_with_content2:
#     print("----------start input message---------------")
#     print(chunk)
#     print("----------end input message---------------")
for idx in true_indices:
    print("----------start input message", idx, "---------------")
    print(splited_md[idx])
    print("----------end input message---------------")

----------start input message 0 ---------------
Threat Brief: Understanding Domain Generation Algorithms (DGA)

[![Logo](/wp-content/uploads/2021/07/PANW_Parent.png)
](https://www.paloaltonetworks.com/)

[![Unit42 Logo](https://unit42.paloaltonetworks.com/wp-content/themes/unit42-v6/dist/images/unit42-logo-white.svg)
](https://unit42.paloaltonetworks.com/)

**

Menu


- [Tools](https://unit42.paloaltonetworks.com/tools/)
- [ATOMs](https://unit42.paloaltonetworks.com/atoms/)
- [Security Consulting](https://www.paloaltonetworks.com/unit42)
- [About Us](https://unit42.paloaltonetworks.com/about-unit-42/)
- [**Under Attack?**](https://start.paloaltonetworks.com/contact-unit42.html)


[English](#)


- [English](https://unit42.paloaltonetworks.com/threat-brief-understanding-domain-generation-algorithms-dga/)
- [Japanese](https://unit42.paloaltonetworks.com/ja/threat-brief
----------end input message---------------
----------start input message 3 ---------------
unit42.paloaltonetworks.com/wp

In [31]:
chunk[0]

'Domain generation algorithm - Wikipedia\n=======================================\n\n[Jump to content](#bodyContent)\n\n\n\t\tMain menu\n\n\nMain menu\tmove to sidebar\thide\n\n\n\t\tNavigation\n\n\n\t\t\t\t\t\t- [Main page](/wiki/Main_Page "Visit the main page [z]")\n- [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")\n- [Current events](/wiki/Portal:Current_events "Articles related to current events")\n- [Random article](/wiki/Special:Random "Visit a randomly selected article [x]")\n- [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia and how it works")\n- [Contact us](//en.wikipedia.org/wiki/Wikipedia:Contact_us "How to contact Wikipedia")\n\n\n\t\tContribute\n\n\n\t\t\t\t\t\t- [Help](/wiki/Help:Contents "Guidance on how to use and edit Wikipedia")\n- [Learn to edit](/wiki/Help:Introduction "Learn how to edit Wikipedia")\n- [Community portal](/wiki/Wikipedia:Community_portal "The hub for editors")\n- [Recent changes](/wiki/Special:RecentChanges "A list 

In [9]:
system_prompt = """
Extract content from the following README chunk.
- Use `+` for factual content.
- Skip menus, links, header, citation and layout.
- If nothing useful, return nothing.
- The inforamtion should only from the chunk
"""
bullet_points = []
for idx in true_indices:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": chunk[idx]}
    ]
    print("----------start input message", idx, "---------------")
    print(splited_md[idx])
    print("----------end input message---------------")
    outputs = pipeline(
        messages,
        max_new_tokens = 512,
        return_full_text=False,
        repetition_penalty=1.2
    )
    bullet_points.append(outputs[0]['generated_text'])
    print(outputs[0])

----------start input message 0 ---------------
Threat Brief: Understanding Domain Generation Algorithms (DGA)

[![Logo](/wp-content/uploads/2021/07/PANW_Parent.png)
](https://www.paloaltonetworks.com/)

[![Unit42 Logo](https://unit42.paloaltonetworks.com/wp-content/themes/unit42-v6/dist/images/unit42-logo-white.svg)
](https://unit42.paloaltonetworks.com/)

**

Menu


- [Tools](https://unit42.paloaltonetworks.com/tools/)
- [ATOMs](https://unit42.paloaltonetworks.com/atoms/)
- [Security Consulting](https://www.paloaltonetworks.com/unit42)
- [About Us](https://unit42.paloaltonetworks.com/about-unit-42/)
- [**Under Attack?**](https://start.paloaltonetworks.com/contact-unit42.html)


[English](#)


- [English](https://unit42.paloaltonetworks.com/threat-brief-understanding-domain-generation-algorithms-dga/)
- [Japanese](https://unit42.paloaltonetworks.com/ja/threat-brief
----------end input message---------------
Nothing useful. 

The extracted information is empty as per your requirements 

In [173]:
def parse_bullet_points(text):
    bullet_pattern = re.compile(r"^([ \t]*)([\-\+\*])\s+(.+)")
    result = []

    for line in text.splitlines():
        match = bullet_pattern.match(line)
        if match:
            indent, symbol, content = match.groups()
            indent_level = len(indent.replace('\t', '    ')) 
            result.append({
                "symbol": symbol,
                "indent": indent_level,
                "content": content.strip()
            })
    return result
def nest_bullets(bullets):
    """
    convert parsed_bullets
    indent < 2 → root; indent ≥ 2 → children
    """
    stack = []      
    root  = []      

    for bullet in bullets:
        item = {**bullet, "children": []}

        while stack and bullet['indent'] <= stack[-1]['indent']:
            stack.pop()

        if bullet['indent'] >= 2 and stack:
            stack[-1]['children'].append(item)
        else:
            root.append(item)

        stack.append(item)

    return root

def build_input_texts(nested_bullets, group_size=4):
    """
    Convert nested_bullets into grouped input_texts.
    - group_size defines the maximum number of lines per input segment (default: 4).
    - For bullets with children:
        • If len(children) > 3 → create separate segments of root + 3 children (root repeats).
        • If len(children) ≤ 3 → keep root and all children together as one unit.
    """
    def fmt_root(b):   # Format root line
        return f"{b['symbol']} {b['content']}"
    def fmt_child(c):  # Format child line (indented with a tab)
        return f"\t{c['symbol']} {c['content']}"

    input_texts = []
    current_group = []

    def flush():
        """Push the current group into the result and clear it."""
        nonlocal current_group
        if current_group:
            input_texts.append('\n'.join(current_group))
            current_group = []

    for bullets in nested_bullets:        # First level list (possibly multiple segments)
        for bullet in bullets:            # Second level: each root bullet
            if bullet['children']:
                kids = bullet['children']

                # -------- case A: children > 3 ⇒ split into chunks of 3 with repeating root --------
                if len(kids) > 3:
                    for i in range(0, len(kids), 3):
                        chunk = kids[i:i + 3]
                        group = [fmt_root(bullet)] + [fmt_child(c) for c in chunk]
                        input_texts.append('\n'.join(group))

                # -------- case B: children ≤ 3 ⇒ keep root and children as a unit --------
                else:
                    block = [fmt_root(bullet)] + [fmt_child(c) for c in kids]

                    # If adding the block would exceed group_size, flush first to keep it intact
                    if len(current_group) + len(block) > group_size:
                        flush()
                    current_group.extend(block)
                    if len(current_group) == group_size:
                        flush()

            # -------- Regular bullet without children --------
            else:
                current_group.append(fmt_root(bullet))
                if len(current_group) == group_size:
                    flush()

    flush()   # Push the remaining lines that didn't form a full group
    return input_texts

input_texts = build_input_texts(nested_bullets=nested_bullets)

parsed_bullets = [parse_bullet_points(bp) for bp in bullet_points]
nested_bullets = [nest_bullets(bp) for bp in parsed_bullets]
text_inputs = build_input_texts(nested_bullets)

In [180]:
system_prompt = """
You are a knowledge graph extraction system. 
Extract key (subject, predicate, object) triples from the following input text. 
Avoid full sentences and auxiliary phrases like "to", "for", "makes it hard", etc.
Use this format:
(Subject, Predicate, Object)
### Example:
Text:
"Apple acquired Beats in 2014 to improve its music streaming services."
Triples:
1. (Apple, acquired, Beats)
2. (Apple, acquired Beats in, 2014)
3. (Apple, aimed to improve, music streaming services)
BTW some information you may find useful:
Title: "Threat Brief: Understanding Domain Generation Algorithms (DGA) - Unit 42"
"""
knowledge_graph = []
for text in text_inputs:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": text}
    ]
    print("----------start input message", idx, "---------------")
    print(text)
    print("----------end input message---------------")
    outputs = pipeline(
        messages,
        max_new_tokens = 512,
        return_full_text=False,
    )
    knowledge_graph.append(outputs[0]['generated_text'])
    print(outputs[0])

----------start input message 63 ---------------
+ A Domain Generation Algorithm (DGA) is a program that generates domain names
+ Attackers used DGAs to automate generating domains for their malware
+ DGAs have made it challenging for defenders to block malicious activities
+ Emerging technologies can help counter DGAs
----------end input message---------------
tools,<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 23 Apr 2025

You are a knowledge graph extraction system. 
Extract key (subject, predicate, object) triples from the following input text. 
Avoid full sentences and auxiliary phrases like "to", "for", "makes it hard", etc.
Use this format:
(Subject, Predicate, Object)
### Example:
Text:
"Apple acquired Beats in 2014 to improve its music streaming services."
Triples:
1. (Apple, acquired, Beats)
2. (Apple, acquired Beats in, 2014)
3. (Apple, aimed to improve, music streaming services)
BTW some information you may fi

In [177]:
search

[{'title': 'Domain generation algorithm - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/Domain_generation_algorithm',
  'body': 'Learn about the malware technique that generates large numbers of domain names to communicate with command and control servers. Find out how DGA works, how to detect it, and what malware families use it.'},
 {'title': 'Threat Brief: Understanding Domain Generation Algorithms (DGA) - Unit 42',
  'href': 'https://unit42.paloaltonetworks.com/threat-brief-understanding-domain-generation-algorithms-dga/',
  'body': "DGA is a technique that malware uses to generate domain names for command and control. Learn how DGA works, why it's a challenge for defenders, and what you can do to prevent it."},
 {'title': 'What is domain generation algorithm (DGA)? - TechTarget',
  'href': 'https://www.techtarget.com/searchsecurity/definition/domain-generation-algorithm-DGA',
  'body': 'A domain generation algorithm (DGA) is a program that generates a large list of domain n